In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,10238
2,Huelva,Confirmados PDIA 14 días,1298
3,Huelva,Tasa PDIA 14 días,"254,13955746823746"
4,Huelva,Confirmados PDIA 7 días,542
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,3
629,Municipio de Huelva sin especificar,Total Confirmados,125
630,Municipio de Huelva sin especificar,Curados,88


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  10238.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  2876.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 951 personas en los últimos 7 días 

Un positivo PCR cada 386 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,10238.0,542.0,1298.0,510743.0,106.119908,254.139557,39.0
Huelva-Costa,5706.0,367.0,859.0,288115.0,127.379692,298.144838,23.0
Condado-Campiña,3145.0,152.0,365.0,155057.0,98.028467,235.397306,16.0
Huelva (capital),2876.0,151.0,372.0,143663.0,105.107091,258.939323,13.0
Palma del Condado (La),477.0,40.0,77.0,10761.0,371.712666,715.546882,10.0
Aljaraque,388.0,25.0,60.0,21260.0,117.591722,282.220132,7.0
Bollullos Par del Condado,234.0,4.0,10.0,14272.0,28.026906,70.067265,2.0
Villalba del Alcor,102.0,9.0,15.0,3338.0,269.622528,449.370881,2.0
Bonares,43.0,3.0,5.0,6058.0,49.521294,82.535490,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Palma del Condado (La),477.0,40.0,77.0,10761.0,371.712666,715.546882,10.0
Lucena del Puerto,82.0,7.0,24.0,3371.0,207.653515,711.954910,0.0
Isla Cristina,376.0,74.0,121.0,21264.0,348.006020,569.036870,1.0
San Bartolomé de la Torre,44.0,10.0,19.0,3736.0,267.665953,508.565310,0.0
Sanlúcar de Guadiana,4.0,1.0,2.0,409.0,244.498778,488.997555,0.0
Lepe,902.0,55.0,132.0,27431.0,200.503080,481.207393,1.0
Villalba del Alcor,102.0,9.0,15.0,3338.0,269.622528,449.370881,2.0
Santa Ana la Real,3.0,1.0,2.0,476.0,210.084034,420.168067,0.0
Palos de la Frontera,214.0,14.0,45.0,11289.0,124.014527,398.618124,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Nerva,87.0,1.0,9.0,5235.0,19.102197,171.919771,0.0,0.111111
Ayamonte,317.0,7.0,59.0,20946.0,33.419269,281.676692,0.0,0.118644
Rociana del Condado,114.0,3.0,18.0,7855.0,38.192234,229.153405,0.0,0.166667
Lucena del Puerto,82.0,7.0,24.0,3371.0,207.653515,711.954910,0.0,0.291667
Palos de la Frontera,214.0,14.0,45.0,11289.0,124.014527,398.618124,0.0,0.311111
Puebla de Guzmán,23.0,1.0,3.0,3073.0,32.541490,97.624471,0.0,0.333333
Cerro de Andévalo (El),37.0,1.0,3.0,2364.0,42.301184,126.903553,0.0,0.333333
Almonte,433.0,3.0,9.0,24191.0,12.401306,37.203919,0.0,0.333333
Sierra de Huelva-Andévalo Central,1263.0,20.0,59.0,67571.0,29.598496,87.315564,-1.0,0.338983
